# System Utilities

This notebook contains various utility functions that are used frequently in the notebooks that implement alignment systems.

In [ ]:
import numpy as np
import os.path

In [ ]:
def verify_scenario_dir(indir):
    '''
    Verifies that the specified scenario directory has the required files.
    
    Inputs
    indir: the scenario directory to verify
    '''
    assert os.path.exists(indir)
    assert os.path.exists(f'{indir}/p.wav')
    assert os.path.exists(f'{indir}/o.mp3')
    assert os.path.exists(f'{indir}/po.mp3')

In [ ]:
def verify_cache_dir(indir):
    '''
    Verifies that the specified cache directory has the required files.
    
    Inputs
    indir: The cache directory to verify
    '''
    assert os.path.exists(f'{indir}/o_chroma.npy')
    assert os.path.exists(f'{indir}/po_chroma.npy')
    assert os.path.exists(f'{indir}/po_o_align.npy')

In [ ]:
def get_orchestra_start(scenario_dir):
    '''
    Determines the start time in the orchestra recording where the query begins.
    
    Inputs
    scenario_dir: the directory containing the scenario information
    
    Returns the query start time in the orchestra recording, specified in seconds
    '''
    
    info_file = f'{scenario_dir}/scenario.info'
    assert os.path.exists(info_file)
    
    with open(info_file, 'r') as f:
        parts = f.readline().split() # last field is start time in orchestra recording
    orch_start_sec = float(parts[-1]) 
    
    return orch_start_sec

In [ ]:
def infer_alignment(wp1, wp2, frames=False, hop=0.02):
    '''
    Given alignments between recording pairs (A, B) and (B, C), infer the estimated alignment
    between the pair (A, C).
    
    Inputs
    wp1: The first warping path, specified as a 2 x N numpy array
    wp2: The second warping path, specified as a 2 x M numpy array
    frames: boolean specifying if the warping path is specified in frames (True) or seconds (False)
    hop: the grid spacing used to perform interpolation
    
    Returns a 2 x L array containing the inferred alignment.
    '''
    
    if frames:
        instants_A = np.arange(wp1[0,-1] + 1) # all A frames
    else:
        instants_A = np.arange(0, wp1[0,-1] + hop, hop) # all A timestamps
    instants_B_interp = np.interp(instants_A, wp1[0,:], wp1[1,:]) # infer corresponding B instants
    instants_C_interp = np.interp(instants_B_interp, wp2[0,:], wp2[1,:]) # infer corresponding C instants
    
    return np.vstack((instants_A, instants_C_interp))